Case Study по модулю SQL
Исполнитель - Мирахмедова Парвина


**Блок 1. Создание схемы и таблиц**

In [ ]:
import json
import pandas as pd
from sqlalchemy import text

from connect import set_connection

SCHEMA = "adw_works"

def create_tables():
    try:
        # читаем содержимое файла 'queries/tables.sql'
        with open('queries/tables.sql') as f:
            tables_query = f.read()
        
        # создаём схему и таблицы
        with set_connection() as psg:
            psg.execute(text(tables_query))
            psg.commit()
        
        print("Tables created successfully")
    except Exception as e:
        print(e)

create_tables()



def read_xl(sheet_name, columns_dict):
    temp_df = pd.read_excel(
        'source/adventure_works.xlsx',
        sheet_name=sheet_name,
        usecols=columns_dict.keys()
    ).rename(columns=columns_dict)
    return temp_df

with open('columns.json') as f:
    tables_dict = json.load(f) 

def insert_to_db(temp_df, tbl_name):
    with set_connection() as psg:
        temp_df.to_sql(
            schema=SCHEMA,
            name=tbl_name,
            con=psg,
            index=False,
            if_exists='append'
        )

def etl(sheet_name, columns_dict, tbl_name):
    print(f"inserting data to {tbl_name}...")
    temp_df = read_xl(sheet_name, columns_dict)
    insert_to_db(temp_df, tbl_name)
    
for k, v in tables_dict.items():
    etl(k, v["columns"], v["table_name"])



**Блок 2. Аналитические задачи**

**Секция 1. Анализ клиентов**
***Сегментация по доходу***: Посчитайте средний годовой личный доход клиентов (YearlyIncome) в разбивке по роду деятельности (Occupation). Итоговая таблица должна содержать следующие поля: occupation, number_of_customers, avg_income.

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import pandas as pd
import plotly.express as px

from connect import set_connection 

query ="""
SELECT 
    occupation,
    COUNT(customer_key) AS number_of_customers,
    round(AVG(yearly_income),2) AS avg_income
FROM 
    adw_works.customers
GROUP BY 
    occupation
ORDER BY 
    occupation;
"""


with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,occupation,number_of_customers,avg_income
0,Clerical,2928,30710.38
1,Management,3075,92325.20
2,Manual,2384,16451.34
3,Professional,5520,74184.78
4,Skilled Manual,4577,51715.10


Вывод.Самый высокий ежегодный доход у представителей менеджмента, а самый низкий - у представителей ручного труда.

**Секция 1. Анализ клиентов**
***Семейный профиль***: Посчитайте долю (в процентах) клиентов с детьми и долю клиентов без детей. Итоговая таблица должна содержать следующие поля: *has_children* (где 1 означает - имеет детей и 0 - не имеет детей), *pct_of_customer_base*.

In [8]:

query ="""SELECT 
    number_of_children,
    round(COUNT(customer_key) * 100.0 / (SELECT COUNT(*) FROM adw_works.customers),2) AS pct_of_customer_base
FROM 
    adw_works.customers
GROUP BY 
    number_of_children
ORDER BY 
    number_of_children;
"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,number_of_children,pct_of_customer_base
0,0,60.14
1,1,13.31
2,2,8.92
3,3,6.51
4,4,5.89
5,5,5.23


Вывод. Покупатели без детей составляют 60% всех покупателей, 40% - покупатели с детьми, где большая часть 13% - это покупатели с одним ребенком.

**Секция 1. Анализ клиентов **
***Высокодоходные клиенты***: Сформируйте список топ 10 клиентов с наибольшей суммой покупок (поле `SalesAmount`). Итоговая таблица должна содержать следующие поля: *customer_key*, *customer_name*, *total_purchase*.

In [11]:
query = """SELECT 
    c.customer_key, 
    c.full_name AS customer_name, 
    round(SUM(s.sales_amount),2) AS total_purchase
FROM 
    adw_works.customers c
JOIN 
    adw_works.sales s ON c.customer_key = s.customer_key
GROUP BY 
    c.customer_key, c.full_name
ORDER BY 
    total_purchase DESC
LIMIT 10;
"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,customer_key,customer_name,total_purchase
0,12301,Nichole Nara,13295.38
1,12132,Kaitlyn Henderson,13294.27
2,12308,Margaret He,13269.27
3,12131,Randall Dominguez,13265.99
4,12300,Adriana Gonzalez,13242.70
5,12321,Rosa Hu,13215.65
6,12124,Brandi Gill,13195.64
7,12307,Brad She,13173.19
8,12296,Francisco Sara,13164.64
9,11433,Maurice Shan,12909.67


Вывод. Наибольшая сумма покупок среди покупателей составляет больше 13 тысяч $, т.е. покупатели готовы тратить не более 13 тысяч.

 **Секция 1. Анализ клиентов**
 ***Влияние семейного положения***: Посчитайте среднюю сумму продаж в разбивке по семейному положению клиентов (*MaritalStatus*) и определите насколько сильно различаются средние суммы между двумя группами. Итоговая таблица должна содержать следующие поля: *year*, *marital_status*, *avg_sales_amount*.

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import pandas as pd
import plotly.express as px

from connect import set_connection 

query = """SELECT 
    cast(EXTRACT(YEAR FROM s.order_date) as INT) AS year,
    c.marital_status,                       
    round(AVG(s.sales_amount),2) AS avg_sales_amount 
FROM 
    adw_works.customers c
JOIN 
    adw_works.sales s ON c.customer_key = s.customer_key
GROUP BY 
    year, c.marital_status
ORDER BY 
    year, marital_status;
    """
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df


intizorparvina


,year,marital_status,avg_sales_amount
0,2001,M,3245.03
1,2001,S,3203.84
2,2002,M,2397.07
3,2002,S,2482.13
4,2003,M,378.56
5,2003,S,427.78
6,2004,M,290.64
7,2004,S,318.05


Вывод. В 2001 и 2002 годах независимо от семейного положения покупатели потратили приблизительно одинаковое количество денег, чуть больше женатые в 2001 и и чуть больше свободные - в 2002 годах. В 2003 и 2004 затраты значительно уменьшились по сравнению с предыдущими годами и при этом, больше стали тратить назамужние/неженатые.

**Секция 2. Анализ продаж** 
 ***Ежемесячные продажи***: Создайте отчёт продаж по месяцам за последние 2 года (2003, 2004). Итоговая таблица должна содержать следующие поля: *year*, *monthkey*, *month_name*, *sales_count* (количество продаж), *sales_amount*.  

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import pandas as pd
import plotly.express as px

from connect import set_connection 

query = """
SELECT
    CAST(EXTRACT(YEAR FROM c.first_purchase_date) AS INT) AS year,                  
    TO_CHAR(c.first_purchase_date, 'YYYYMM') AS monthkey,               
    TO_CHAR(c.first_purchase_date, 'FMMonth') AS month_name,            
    COUNT(*) AS sales_count,                                             
    round(SUM(s.sales_amount),1) AS sales_amount                                  
FROM
    adw_works.customers c                                                
JOIN
    adw_works.sales s ON c.customer_key = s.customer_key                
WHERE
    EXTRACT(YEAR FROM c.first_purchase_date) IN (2003, 2004)            
GROUP BY
    EXTRACT(YEAR FROM c.first_purchase_date),                            
    TO_CHAR(c.first_purchase_date, 'YYYYMM'),                           
    TO_CHAR(c.first_purchase_date, 'FMMonth')                           
ORDER BY
    year, monthkey;                                                     
                                             
"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,year,monthkey,month_name,sales_count,sales_amount
0,2003,200301,January,1067,949602.8
1,2003,200302,February,1152,1006238.4
2,2003,200303,March,1191,1009850.7
3,2003,200304,April,1109,971965.8
4,2003,200305,May,1324,1161483.1
5,2003,200306,June,1253,1183195.9
6,2003,200307,July,585,343439.5
7,2003,200308,August,5346,397916.9
8,2003,200309,September,3613,377478.2
9,2003,200310,October,3463,436072.9


Вывод. Наибольшее количество продаж в 2003 году отмечалось в августе, а наименьшее - в июле. В 2004 - наибольшее количество продаж отмечалось в мае, а наименьшее - также в июле.

**Секция 2. Анализ продаж** 
 ***Продажи по регионам***: Посчитайте сумму продаж в разбивке по регионам. Итоговая таблица должна содержать следующие поля: *region*, *sales_count*, *sales_amount*.

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import pandas as pd
import plotly.express as px

from connect import set_connection 
query = """ 
SELECT 
    t.country AS region,
    COUNT(s.sales_amount) AS sales_count,
    round(SUM(s.sales_amount),1) AS sales_amount
FROM 
    adw_works.sales s
JOIN 
    adw_works.territory t ON s.territory_key = t.territory_key
GROUP BY 
    t.country
ORDER BY 
    t.country;

"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,region,sales_count,sales_amount
0,Australia,13345,9061000.6
1,Canada,7620,1977844.9
2,France,5558,2644017.7
3,Germany,5625,2894312.3
4,United Kingdom,6906,3391712.2
5,United States,21344,9389789.5


Вывод. Больше всего покупок из США и их вклад в сумму продаж самый большой, наименьший вклад вносят покупки из Франции,их же и наименьшее количество.

### Секция 3. Анализ продуктов  
**Доля продаж**: Посчитайте какую долю от общих продаж составляет каждая категория продуктов. Итоговая таблица должна содержать следующие поля: *year*, *product_key*, *product_category_key*, *english_product_category_name*, *sales_amount*, *pct_of_total_sales*.

In [26]:
query = """ 

SELECT 
    CAST(EXTRACT(YEAR FROM p.start_date) AS INT) AS year,
    p.product_key,
    pc.category_key AS product_category_key,
    pc.category_name AS english_product_category_name,
    ROUND(SUM(s.sales_amount),1) AS sales_amount,
    ROUND((SUM(s.sales_amount) / total_sales.total_sales_amount) * 100,2) AS pct_of_total_sales
FROM 
    adw_works.products p
JOIN 
    adw_works.sales s ON p.product_key = s.product_key
JOIN 
    adw_works.product_subcategory ps ON p.subcategory_key = ps.subcategory_key
JOIN 
    adw_works.product_category pc ON ps.category_key = pc.category_key
JOIN 
    (SELECT SUM(sales_amount) AS total_sales_amount
     FROM adw_works.sales) total_sales ON 1=1
GROUP BY 
    CAST(EXTRACT(YEAR FROM p.start_date) AS INT), p.product_key, pc.category_key, pc.category_name, total_sales.total_sales_amount
ORDER BY 
    year, pc.category_key, p.product_key;


"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,year,product_key,product_category_key,english_product_category_name,sales_amount,pct_of_total_sales
0,2001,310,1,Bikes,1202298.7,4.10
1,2001,311,1,Bikes,1005493.9,3.42
2,2001,312,1,Bikes,1205877.0,4.11
3,2001,313,1,Bikes,1080637.5,3.68
4,2001,314,1,Bikes,1055589.7,3.60
...,...,...,...,...,...,...
153,2003,537,4,Accessories,48860.0,0.17
154,2003,538,4,Accessories,22435.6,0.08
155,2003,539,4,Accessories,23140.7,0.08
156,2003,540,4,Accessories,27970.8,0.10


Вывод. Самый большой процент от продаж независимо от года составляют велосипеды. На втором месте - аксессуары, одежда на третьем месте. Товары из категории 2 (запчасти к велосипедам и велооборудование) не продавались.  

**Секция 3. Анализ продуктов**
***Самые продаваемые продукты***: Определите топ 5 продуктов с наибольшей суммой продаж. Итоговая таблица должна содержать следующие поля: *product_key*, *product_name*, *english_product_category_name*, *sales_amount*

In [28]:
query = """ 
SELECT 
    p.product_key,
    p.product_name,
    pc.category_name AS english_product_category_name,
    ROUND(SUM(s.sales_amount),1) AS sales_amount
FROM 
    adw_works.products p
JOIN 
    adw_works.sales s ON p.product_key = s.product_key
JOIN 
    adw_works.product_subcategory ps ON p.subcategory_key = ps.subcategory_key
JOIN 
    adw_works.product_category pc ON ps.category_key = pc.category_key
GROUP BY 
    p.product_key, p.product_name, pc.category_name
ORDER BY 
    sales_amount DESC
LIMIT 5;

"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,product_key,product_name,english_product_category_name,sales_amount
0,312,"Road-150 Red, 48",Bikes,1205877.0
1,310,"Road-150 Red, 62",Bikes,1202298.7
2,313,"Road-150 Red, 52",Bikes,1080637.5
3,314,"Road-150 Red, 56",Bikes,1055589.7
4,311,"Road-150 Red, 44",Bikes,1005493.9


Вывод: в ТОП -5 входят 5 моделей дорожных велосипедов Road -150 Red.

 **Секция 3. Анализ продуктов**
 ***Маржа от продаж***: Посчитайте разницу между суммой продаж (*SalesAmount*) за минусом себестоимости (*TotalProductCost*), налогов (*TaxAmt) и расходов на доставку (*Freight*) по каждому продукту в разбивке по годам и месяцам. Итоговая таблица должна содержать следующие поля: *year*, *monthkey*, *month_name*, *product_key*, *product_name*, *sales_amount*, *total_product_cost*, *tax_amt*, *freight*, *margin*, *margin_pct* (маржа как процент от суммы продаж).

In [30]:
query = """ 
SELECT 
    CAST(EXTRACT(YEAR FROM c.first_purchase_date) AS INT) AS year,                  
    TO_CHAR(c.first_purchase_date, 'YYYYMM') AS monthkey,               
    TO_CHAR(c.first_purchase_date, 'FMMonth') AS month_name,            
    s.product_key, 
    p.product_name, 
    round(SUM(s.sales_amount), 1) AS sales_amount,
    round(SUM(s.total_product_cost), 1) AS total_product_cost,
    round(SUM(s.tax_amount), 1) AS tax_amt,
    round(SUM(s.freight), 1) AS freight,
    round(SUM(s.sales_amount) - SUM(s.total_product_cost) - SUM(s.tax_amount) - SUM(s.freight), 1) AS margin,
    round((SUM(s.sales_amount) - SUM(s.total_product_cost) - SUM(s.tax_amount) - SUM(s.freight)) / SUM(s.sales_amount) * 100, 2) AS margin_pct
FROM
    adw_works.customers c
JOIN 
    adw_works.sales s ON c.customer_key = s.customer_key
JOIN
    adw_works.products p ON s.product_key = p.product_key
GROUP BY
    year, monthkey, month_name, s.product_key, p.product_name
ORDER BY 
    year DESC, monthkey DESC, s.product_key;

"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,year,monthkey,month_name,product_key,product_name,sales_amount,total_product_cost,tax_amt,freight,margin,margin_pct
0,2004,200407,July,214,"Sport-100 Helmet, Red",1539.6,575.8,123.2,38.5,802.1,52.10
1,2004,200407,July,217,"Sport-100 Helmet, Black",1959.4,732.8,156.8,49.0,1020.9,52.10
2,2004,200407,July,222,"Sport-100 Helmet, Blue",2204.4,824.4,176.3,55.1,1148.5,52.10
3,2004,200407,July,225,AWC Logo Cap,539.4,415.3,43.2,13.5,67.4,12.50
4,2004,200407,July,228,"Long-Sleeve Logo Jersey, S",899.8,692.9,72.0,22.5,112.5,12.50
...,...,...,...,...,...,...,...,...,...,...,...
3947,2001,200107,July,599,"Mountain-500 Black, 48",540.0,294.6,43.2,13.5,188.7,34.95
3948,2001,200107,July,600,"Mountain-500 Black, 52",540.0,294.6,43.2,13.5,188.7,34.95
3949,2001,200107,July,604,"Road-750 Black, 44",2160.0,1374.6,172.8,54.0,558.6,25.86
3950,2001,200107,July,605,"Road-750 Black, 48",540.0,343.6,43.2,13.5,139.6,25.86


Вывод.Подсчитана маржа по каждому продукту с разбивкой мо месяцам и годам.